# Library

In [9]:
# Installation
%pip install --quiet mesa

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Import
import mesa
import numpy as np
import pandas as pd

import math

# Helper Functions

# Class Definition

## Agent

In [13]:
class ChartistAgent(mesa.Agent):
    def __init__(self, id, model, fiat_owned, bitcoin_owned, chartist_day_reference):
        super().__init__(id, model)

        # Attribute Initialization         
        self.fiat = fiat_owned
        self.bitcoin = bitcoin_owned
        self.is_close = True
        self.n = chartist_day_reference
        
        # Default Rule : EMA for both opening and closing
        self.is_open_filtering_high = False
        self.is_close_filtering_high = False
        
        if (np.random.uniform() <= 0.25):
            # Rule : Filtering for both opening and closing             
            self.is_open_filtering_high = True
            self.is_close_filtering_high = True
        elif (np.random.uniform() > 0.25 and np.random.uniform() < 0.5):
            # Rule : Filtering for opening and EMA for closing
            self.is_open_filtering_high = True
        elif (np.random.uniform() > 0.25 and np.random.uniform() < 0.75):
            # Rule : EMA for opening and Filtering for closing
            self.is_close_filtering_high = True
    
    def step(self):
        if self.is_close:
            # Potential Buy
            filtering_value = self.calculate_avg_n_days(self.model.price_history, n)
            ema_value = self.calculate_ema_n_days(self.model.price_history, n)
            
            probability_to_buy = ()

            if (filtering_value < self.model.today_price):
                self.is_open = False
                
                if self.bitcoin > 0:
                    price = self.model.today_price

                    self.bitcoin -= 1
                    self.fiat += price
                    
                    self.model.today_volume -= 1
                    print(f'{self.unique_id} : fiat {self.fiat} bitcoin {self.bitcoin}')
        else:
            print(f'potential buy for {self.unique_id}')
            filtering_value = calculate_avg_n_days(self.model.price_history, n)
            print(f'{filtering_value} vs {self.model.today_price}')

            if (filtering_value > self.model.today_price):
                self.is_open = True
                
                if self.fiat > 0:
                    price = self.model.today_price

                    self.bitcoin += 1
                    self.fiat -= price
                    
                    self.model.today_volume += 1
                    print(f'{self.unique_id} : fiat {self.fiat} bitcoin {self.bitcoin}')
                    
    # Agent Helper Function
    # Filtering Rule
    def calculate_avg_n_days(price_history, n):
        start_index = max(0, len(price_history) - n)
        window = price_history[start_index:len(price_history)]

        return sum(window) / len(window)

    # EMA Rule
    def calculate_ema_n_days(price_history, n):
        smooth_factor = 2 / (n + 1)
        ema = [price_history[0]]

        for i in range(1, len(price_history)):
            value = smooth_factor * price_history[i] + (1 - smooth_factor) * ema[i-1]
            ema.append(value)

        return ema[len(ema)-1]

    # Prospect Theory Calculator
    def calculate_prospect_theory_value(self, pnl):
        value = 0
        total_value = 0
        if pnl >= 0:
            value = pnl ** self.prospect_theory_alpha
            hypothetical_complimentary_value = -1 * self.prospect_theory_lambda * ((-1 * pnl) ** self.prospect_theory_alpha)
            total_value = value + hypothetical_complimentary_value
        else:
            value = -1 * self.prospect_theory_lambda * ((-1 * pnl) ** self.prospect_theory_alpha)
            hypothetical_complimentary_value = pnl ** self.prospect_theory_alpha
            total_value = value + hypothetical_complimentary_value

        return value, total_value

# Bitcoin Market Model

In [14]:
import random

class BitcoinMarketModel(mesa.Model):
    def __init__(self, t_start, price_start, agent_calculator_a, agent_calculator_b, total_fiat, total_bitcoin, chartist_ratio, chartist_day_reference, computational_factor):
        self.t = t_start
        self.price = price_start
        self.price_history = [price_start]
        
        self.agent_calculator_a = agent_calculator_a
        self.agent_calculator_b = agent_calculator_b
        self.number_of_agents = calculate_number_of_agents()
        self.total_fiat = total_fiat
        self.total_bitcoin = total_bitcoin
        self.supply = 0
        self.demand = 0
        
        self.computational_factor = computational_factor

        for i in range(self.number_of_agents):
            fiat_owned = spread_wealth_fiat(self, i)
            bitcoin_owned = spread_wealth_bitcoin(self, i)
            
            if (np.random.uniform() <= 0.6):
                a = ChartistAgent(str(f"chartist-{i}"), 
                              self, 
                              fiat_owned,
                              bitcoin_owned, 
                              chartist_day_reference)

                self.schedule.add(a)
            else:
                # Create Random Agent              
    
    def step(self):
        # Before Stepping
        self.price += self.calculate_today_price_change(self)
        self.supply = 0
        self.demand = 0
        
        # Stepping
        self.schedule.step()
        
        # After Stepping
        self.price_history.append(self.price)

        
    # Model Helper Function
    # Calculate Number of Agents
    def calculate_number_of_agents(self):
        return self.agent_calculator_a * math.exp(self.agent_calculator_b * self.t) / self.computational_factor
    
    # Calculate Price
    def calculate_today_price_change(self):
        supply_demand_difference = self.demand - self.supply
        sign = math.copysign(1, supply_demand_difference)
        
        return math.floor((math.sqrt(2) / 2) *  sign * math.sqrt(abs(supply_demand_difference)))
    
    # Wealth Distribution
    def spread_wealth_fiat(self, n):
        return self.total_fiat * (n ** (-1 / self.zipf_alpha))
    
    def spread_wealth_bitcoin(self, n):
        return self.total_bitcoin * (n ** (-1 / self.zipf_alpha))
    
    # Calculate New Bitcoin Introduced to Model
    def calculate_new_bitcoin(self):
        return 0.6 * self.total_bitcoin

# Simulation

In [15]:
# Model Parameters
# total_bitcoin = 18200000 # https://www.statista.com/statistics/247280/number-of-bitcoins-in-circulation/
total_bitcoin = 10000 
tradable_bitcoin_ratio = 0.7 # https://www.makeuseof.com/how-much-bitcoin-is-lost-forever/#:~:text=Estimates%20suggest%20that%20around%206,a%20guess%20at%20the%20password.
initial_price = 10

# Model Derivative Parameters
tradable_bitcoin = total_bitcoin * tradable_bitcoin_ratio

# Agents Parameters
agents = 2
n = 3
chartist_ratio = 100 # based on Cocco's paper
random_ratio = 40
fiat_owned = 100

a = 17440
b = 0.002465

In [16]:
model = BitcoinMarketModel(agents, 
                           chartist_ratio, 
                           random_ratio, 
                           fiat_owned,
                           tradable_bitcoin, 
                           initial_price, 
                           n)

for i in range(10):
    model.step()

agents = model.schedule.agents
for a in agents:
    print(f"Agent {a.unique_id} : Fiat {a.fiat} Bitcoin {a.bitcoin}")

new cycle
potential buy for chartist-1
[10]
10.0 vs 11
potential buy for chartist-0
[10]
10.0 vs 11
vol = 0
new cycle
potential buy for chartist-0
[10, 11]
10.5 vs 11
potential buy for chartist-1
[10, 11]
10.5 vs 11
vol = 0
new cycle
potential buy for chartist-0
[10, 11, 11]
10.666666666666666 vs 11
potential buy for chartist-1
[10, 11, 11]
10.666666666666666 vs 11
vol = 0
new cycle
potential buy for chartist-1
[11, 11, 11]
11.0 vs 11
potential buy for chartist-0
[11, 11, 11]
11.0 vs 11
vol = 0
new cycle
potential buy for chartist-0
[11, 11, 11]
11.0 vs 12
potential buy for chartist-1
[11, 11, 11]
11.0 vs 12
vol = 0
new cycle
potential buy for chartist-0
[11, 11, 12]
11.333333333333334 vs 12
potential buy for chartist-1
[11, 11, 12]
11.333333333333334 vs 12
vol = 0
new cycle
potential buy for chartist-0
[11, 12, 12]
11.666666666666666 vs 11
chartist-0 : fiat 89 bitcoin 3501.0
potential buy for chartist-1
[11, 12, 12]
11.666666666666666 vs 11
chartist-1 : fiat 89 bitcoin 3501.0
vol = 2
